In [1]:
%pip install natsort tqdm transformers torch torchvision


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path
import glob
import re
import shutil

from transformers import CLIPProcessor, CLIPModel
import torch
from torchvision.io import decode_image
from natsort import natsorted
from tqdm import tqdm

2024-10-30 13:54:54.354278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-30 13:54:54.354297: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
DATASET_PATH = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'original_split', '**')
POSITIVE_PATH = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'forest')
NEGATIVE_PATH = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'urban')  # for visual control

for path in (POSITIVE_PATH, NEGATIVE_PATH):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.mkdir(path)
    
POSITIVE_LABELS = ['forest', 'steppe', 'fields', 'snowy forest', 'snowy steppe', 'woods', 'taiga', 'few cars', 'road', 
                   'many green trees', 'big trees', 'grass', 'bareland', 'grassland', 'farmland', 'river', 'lake', 
                   'hills', 'highway', 'runway', 'tree', 'meadow', 'swamp', 'bushes', 'dead bushes', 'dead tree', 
                   'person', 'pedestrian', 'human', 'shack', 'parking lot', 'suburban area']
NEGATIVE_LABELS = ['city', 'town', 'several buildings', 'many buildings', 'aiport', 'many houses', 'terminal', 
                   'building', 'house', 'urban area']
LABELS = POSITIVE_LABELS + NEGATIVE_LABELS
TEXT_QUERIES = [f'an aerial photograph of {label}' for label in LABELS]

In [4]:
device = torch.device('cuda')
# device = torch.device('cpu')

model = CLIPModel.from_pretrained("flax-community/clip-rsicd").to(device)
processor = CLIPProcessor.from_pretrained("flax-community/clip-rsicd")

In [5]:
images = [file for file in glob.glob(os.path.join(DATASET_PATH, '*'), recursive=True) if re.match(r'(.*\.jpg)', file)]
annotations = [file for file in glob.glob(os.path.join(DATASET_PATH, '*'), recursive=True) if re.match(r'(.*\.txt)', file)]

images = natsorted(images)
annotations = natsorted(annotations)

len(images), len(annotations)

(265277, 265277)

In [6]:
stems  = []
for image, annotation in tqdm(zip(images, annotations)):
    if Path(image).stem != Path(annotation).stem:
        raise ValueError('Images and annotations indices do not match')
    
    T = decode_image(image)
    
    X = processor(text=TEXT_QUERIES, images=T, return_tensors='pt', padding=True)
    Y = model(**X.to(device))
    
    label = LABELS[torch.argmax(Y.logits_per_image.softmax(dim=1).cpu().detach()[0])]
    if label in POSITIVE_LABELS:
        for file in (image, annotation):
            shutil.copy(file, POSITIVE_PATH)
    else:
        for file in (image, annotation):
            shutil.copy(file, NEGATIVE_PATH)

265277it [1:09:43, 63.42it/s]
